# 🔥 How to use a Computer Vision Model with Focoos

🐍 Setup Focoos

# 🎨 There are three ways to use a model:

1. Use it on the Focoos' efficient servers with the RemoteModel
2. Use the model in PyTorch
3. Use the exported optimized version of the model using a supported inference runtime.

## 🐍 Connect with Focoos

In [ ]:
from focoos.hub import FocoosHUB

FOCOOS_API_KEY = None  # write here your API key
hub = FocoosHUB(api_key=FOCOOS_API_KEY)

Let's also download a sample image to test the model.

In [ ]:
from io import BytesIO

import requests
from PIL import Image

# Download the image
url = "https://public.focoos.ai/samples/pexels-abby-chung.jpg"
response = requests.get(url)
image = Image.open(BytesIO(response.content))

## 📦 See your models
You can see the models available for you on the platform with an intuitive user interface.
However, you can also list them using the Hub functionalities.

In [ ]:
from pprint import pprint

pprint(hub.list_remote_models())

## 🌍 Remote Inference

In this section, you'll run a model on the Focoos' servers instead of on your machine. The image will be packed and sent on the network to the servers, where it is processed and the results is retured to your machine, all in few milliseconds. 

In [ ]:
model_ref = "fai-detr-l-obj365"  # use any of your models here

model = hub.get_remote_model(model_ref)

Using the model is as simple as it could! Just call it with an image.

In [ ]:
detections = model(image)
pprint(detections)

If you want to visualize the result on the image, there's a utily for you.

In [ ]:
from focoos.utils.vision import annotate_image

display(annotate_image(image, detections, task=model.model_info.task, classes=model.model_info.classes))

## 🔥 Torch Inference

This section demonstrates how to perform local inference using a plain Pytorch model.
We will load a model and then run inference on a sample image.

First, let's get a model. We need to use the `ModelManager` that will take care of instaciating the right model starting from a model reference (for example, the `hub://fai-detr-l-obj365`) 

In [ ]:
from focoos.model_manager import ModelManager

model_ref = "hub://fai-detr-l-obj365"  # use any of your models here

model = ModelManager.get(model_ref)

You can now run the model by simply passing it an image

In [ ]:
from pprint import pprint

detections = model(image)
pprint(detections)

and visualize the results using the annotate_image utility.

In [ ]:
from focoos.utils.vision import annotate_image

display(annotate_image(image, detections, task=model.model_info.task, classes=model.model_info.classes))

How fast is this model locally? We can compute it's speed by using the benchmark utility.

In [ ]:
model.benchmark(iterations=10, size=640)

## 🔨 Optimized Inference

As you can see, using the torch model is great, but we can achieve better performance by exporting and running it with a optimized runtime, such as Torchscript, TensorRT, CoreML or the ones available on ONNXRuntime.

In the following cells, we will export the previous model for one of these and run it.

### Torchscript

We already provide multiple inference runtime, that you can see on the `RuntimeTypes` enum. Let's select Torchscript as an example.

In [ ]:
from focoos.ports import RuntimeType

runtime = RuntimeType.TORCHSCRIPT_32

It's time to export the model. We can use the export method of the models.

In [ ]:
optimized_model = model.export(runtime_type=runtime, image_size=1024)

Let's visualize the output. As you will see, there are not differences from the model in pure torch.

In [ ]:
from focoos.utils.vision import annotate_image

detections = optimized_model(image)
display(annotate_image(image, detections, task=model.model_info.task, classes=model.model_info.classes))

But, let's see its latency! 

In [ ]:
optimized_model.benchmark(iterations=10, size=640)

Wow! That's a lot faster! And without losing a bit in performance!

You can also try different runtimes. Please note that you need to install the relative packages for onnx and tensorRT for using them.

### ONNX with CUDA

In [ ]:
from focoos.ports import RuntimeType
from focoos.utils.vision import annotate_image

runtime = RuntimeType.ONNX_CUDA32
optimized_model = model.export(runtime_type=runtime)

detections = optimized_model(image)
display(annotate_image(image, detections, task=model.model_info.task, classes=model.model_info.classes))

optimized_model.benchmark(iterations=10, size=640)

### ONNX with TensorRT

In [ ]:
from focoos.ports import RuntimeType
from focoos.utils.vision import annotate_image

runtime = RuntimeType.ONNX_TRT16
optimized_model = model.export(runtime_type=runtime)

detections = optimized_model(image)
display(annotate_image(image, detections, task=model.model_info.task, classes=model.model_info.classes))

optimized_model.benchmark(iterations=10, size=640)